In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from tqdm import tqdm
import pickle
import json

import plotly.graph_objects as go

# Custom RNN Cell
class CustomRNNCell(layers.Layer):
    def __init__(self, units, **kwargs):
        super(CustomRNNCell, self).__init__(**kwargs)
        self.units = units
        self.state_size = units

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                    initializer='glorot_uniform',
                                    name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='orthogonal',
            name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units,),
                                  initializer='zeros',
                                  name='bias')

    def call(self, inputs, states):
        prev_output = states[0]
        h = tf.matmul(inputs, self.kernel)
        output = tf.nn.tanh(h + tf.matmul(prev_output, self.recurrent_kernel) + self.bias)
        return output, [output]

# Custom Optimizer
class CustomAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, **kwargs):
        super(CustomAdam, self).__init__(
            learning_rate=learning_rate,
            beta_1=beta_1,
            beta_2=beta_2,
            epsilon=epsilon,
            **kwargs
        )

# Data preprocessing function
def extract_features(file_path, max_length=128):
    try:
        audio, sr = librosa.load(file_path, sr=22050)
        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        # Extract spectral features
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)
        
        # Combine features
        features = np.vstack([mfccs, spectral_centroids, spectral_rolloff, zero_crossing_rate])
        
        # Pad or truncate to fixed length
        if features.shape[1] < max_length:
            features = np.pad(features, ((0, 0), (0, max_length - features.shape[1])), mode='constant')
        else:
            features = features[:, :max_length]
            
        return features.T  # Shape: (max_length, n_features)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Load data from folders - Surah An-Naba Dataset
def load_data():
    """
    Load audio data dari multiple pembaca Quran (Surah An-Naba)
    
    Dataset Structure:
    - sample_1/, sample_2/, ..., sample_7/ = 7 pembaca berbeda
    - Setiap folder berisi 41 files: 078000.mp3 sampai 078040.mp3
    - 078000.mp3 = Bismillah (ayat pembuka) -> label 0
    - 078001.mp3 = Ayat 1 -> label 1
    - ...
    - 078040.mp3 = Ayat 40 -> label 40
    
    Total: 7 pembaca × 41 ayat = 287 audio samples
    """
    base_path = r"d:\new_project\quran_detect"
    
    # Cek semua folder sample yang tersedia (sample_1 hingga sample_7)
    available_folders = []
    for i in range(1, 8):  # sample_1 sampai sample_7
        folder_name = f'sample_{i}'
        folder_path = os.path.join(base_path, folder_name)
        if os.path.exists(folder_path):
            available_folders.append(folder_name)
        else:
            print(f"Warning: Folder {folder_name} not found")
    
    print(f"Found {len(available_folders)} sample folders: {available_folders}")
    
    X = []
    y = []
    file_info = []  # Track file information for debugging
    
    for folder in available_folders:
        folder_path = os.path.join(base_path, folder)
        print(f"Processing {folder}...")
        
        files = sorted([f for f in os.listdir(folder_path) if f.endswith('.mp3')])
        print(f"  Found {len(files)} audio files")
        
        # Validasi bahwa file sesuai format yang diharapkan
        expected_files = [f"078{i:03d}.mp3" for i in range(41)]  # 078000.mp3 to 078040.mp3
        missing_files = [f for f in expected_files if f not in files]
        if missing_files:
            print(f"  Warning: Missing files in {folder}: {missing_files[:5]}...")
        
        for file in tqdm(files, desc=f"Processing {folder}"):
            file_path = os.path.join(folder_path, file)
            features = extract_features(file_path)
            
            if features is not None:
                X.append(features)
                
                # Extract verse number from filename
                # 078000.mp3 -> 0 (Bismillah)
                # 078001.mp3 -> 1 (Ayat 1)
                # 078040.mp3 -> 40 (Ayat 40)
                verse_num = int(file.split('.')[0][-3:])
                y.append(verse_num)
                
                # Track file info for debugging
                file_info.append({
                    'folder': folder,
                    'filename': file,
                    'verse_label': verse_num
                })
            else:
                print(f"  Failed to extract features from: {file}")
    
    X = np.array(X)
    y = np.array(y)
    
    # Print dataset summary
    print(f"\n=== Dataset Summary ===")
    print(f"Total samples: {len(X)}")
    print(f"Total pembaca: {len(available_folders)}")
    print(f"Feature shape per sample: {X[0].shape if len(X) > 0 else 'N/A'}")
    print(f"Verse range: {min(y)} to {max(y)}")
    print(f"Unique verses: {len(np.unique(y))}")
    
    # Cek distribusi ayat
    verse_counts = np.bincount(y)
    print(f"Samples per verse (should be {len(available_folders)} each):")
    for verse_id in range(len(verse_counts)):
        if verse_counts[verse_id] > 0:
            verse_name = "Bismillah" if verse_id == 0 else f"Ayat {verse_id}"
            print(f"  {verse_name} (ID:{verse_id}): {verse_counts[verse_id]} samples")
    
    return X, y, file_info

# Custom RNN Model
def create_custom_rnn_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Custom RNN layers
    rnn_cell1 = CustomRNNCell(128)
    rnn_layer1 = layers.RNN(rnn_cell1, return_sequences=True)(inputs)
    rnn_layer1 = layers.Dropout(0.3)(rnn_layer1)
    
    rnn_cell2 = CustomRNNCell(64)
    rnn_layer2 = layers.RNN(rnn_cell2, return_sequences=False)(rnn_layer1)
    rnn_layer2 = layers.Dropout(0.3)(rnn_layer2)
    
    # Dense layers
    dense1 = layers.Dense(256, activation='relu')(rnn_layer2)
    dense1 = layers.Dropout(0.4)(dense1)
    dense2 = layers.Dense(128, activation='relu')(dense1)
    dense2 = layers.Dropout(0.3)(dense2)
    
    outputs = layers.Dense(num_classes, activation='softmax')(dense2)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Function to save model and metadata
def save_model_and_metadata(model, label_encoder, history, model_dir="model_saves"):
    """
    Simpan model, label encoder, dan metadata training
    """
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # Save model
    model_path = os.path.join(model_dir, "quran_verse_model.h5")
    model.save(model_path)
    print(f"Model saved to: {model_path}")
    
    # Save label encoder
    encoder_path = os.path.join(model_dir, "label_encoder.pkl")
    with open(encoder_path, 'wb') as f:
        pickle.dump(label_encoder, f)
    print(f"Label encoder saved to: {encoder_path}")
    
    # Save training history
    history_path = os.path.join(model_dir, "training_history.pkl")
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    print(f"Training history saved to: {history_path}")
    
    # Save metadata
    metadata = {
        "num_classes": len(label_encoder.classes_),
        "verse_labels": label_encoder.classes_.tolist(),
        "input_shape": model.input_shape[1:],  # Remove batch dimension
        "total_epochs": len(history.history['loss']),
        "final_accuracy": history.history['val_accuracy'][-1],
        "final_loss": history.history['val_loss'][-1]
    }
    
    metadata_path = os.path.join(model_dir, "model_metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Metadata saved to: {metadata_path}")

# Function to load model and metadata
def load_model_and_metadata(model_dir="model_saves"):
    """
    Muat model, label encoder, dan metadata yang telah disimpan
    """
    try:
        # Load model
        model_path = os.path.join(model_dir, "quran_verse_model.h5")
        model = tf.keras.models.load_model(model_path, custom_objects={
            'CustomRNNCell': CustomRNNCell,
            'CustomAdam': CustomAdam
        })
        print(f"Model loaded from: {model_path}")
        
        # Load label encoder
        encoder_path = os.path.join(model_dir, "label_encoder.pkl")
        with open(encoder_path, 'rb') as f:
            label_encoder = pickle.load(f)
        print(f"Label encoder loaded from: {encoder_path}")
        
        # Load metadata
        metadata_path = os.path.join(model_dir, "model_metadata.json")
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        print(f"Metadata loaded from: {metadata_path}")
        
        # Load training history if needed
        history_path = os.path.join(model_dir, "training_history.pkl")
        history = None
        if os.path.exists(history_path):
            with open(history_path, 'rb') as f:
                history = pickle.load(f)
            print(f"Training history loaded from: {history_path}")
        
        return model, label_encoder, metadata, history
        
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None, None

# Function to predict single audio file
def predict_verse(model, label_encoder, audio_file_path):
    """
    Prediksi nomor ayat dari file audio tunggal
    
    Returns:
        verse_number (int): Nomor ayat (0=Bismillah, 1-40=Ayat 1-40)
        confidence (float): Confidence score (0-1)
    """
    # Extract features from audio file
    features = extract_features(audio_file_path)
    if features is None:
        return None, None
    
    # Reshape for prediction
    features = features.reshape(1, features.shape[0], features.shape[1])
    
    # Make prediction
    prediction = model.predict(features, verbose=0)
    predicted_class = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    # Convert back to original label
    verse_number = label_encoder.inverse_transform([predicted_class])[0]
    
    return verse_number, confidence

def get_verse_name(verse_number):
    """
    Konversi nomor ayat ke nama yang mudah dibaca
    
    Args:
        verse_number (int): 0=Bismillah, 1-40=Ayat 1-40
    
    Returns:
        str: Nama ayat yang mudah dibaca
    """
    if verse_number == 0:
        return "Bismillah (Pembuka)"
    elif 1 <= verse_number <= 40:
        return f"Ayat {verse_number}"
    else:
        return f"Unknown ({verse_number})"

def predict_verse_with_details(model, label_encoder, audio_file_path):
    """
    Prediksi dengan output yang lebih detail dan mudah dibaca
    """
    verse_number, confidence = predict_verse(model, label_encoder, audio_file_path)
    
    if verse_number is None:
        return None
    
    verse_name = get_verse_name(verse_number)
    
    result = {
        'verse_number': verse_number,
        'verse_name': verse_name,
        'confidence': confidence,
        'confidence_percent': confidence * 100,
        'file_path': audio_file_path,
        'filename': os.path.basename(audio_file_path)
    }
    
    return result

# Function to test on a folder of audio files
def test_on_folder(model, label_encoder, test_folder_path):
    """
    Test model pada folder berisi file audio dan tampilkan hasil
    """
    if not os.path.exists(test_folder_path):
        print(f"Test folder not found: {test_folder_path}")
        return None
    
    results = []
    files = sorted([f for f in os.listdir(test_folder_path) if f.endswith('.mp3')])
    
    print(f"Testing on {len(files)} files from {test_folder_path}")
    
    for file in tqdm(files):
        file_path = os.path.join(test_folder_path, file)
        
        # Get actual verse number from filename
        actual_verse = int(file.split('.')[0][-3:])
        
        # Predict
        predicted_verse, confidence = predict_verse(model, label_encoder, file_path)
        
        if predicted_verse is not None:
            # Get verse names for better readability
            actual_verse_name = get_verse_name(actual_verse)
            predicted_verse_name = get_verse_name(predicted_verse)
            
            results.append({
                'filename': file,
                'actual_verse': actual_verse,
                'actual_verse_name': actual_verse_name,
                'predicted_verse': predicted_verse,
                'predicted_verse_name': predicted_verse_name,
                'confidence': confidence,
                'correct': actual_verse == predicted_verse
            })
    
    # Calculate accuracy
    if results:
        accuracy = sum([r['correct'] for r in results]) / len(results)
        print(f"\nTest Results:")
        print(f"Total files tested: {len(results)}")
        print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"Correct predictions: {sum([r['correct'] for r in results])}")
        print(f"Wrong predictions: {sum([not r['correct'] for r in results])}")
        
        # Show some examples
        print(f"\nFirst 10 predictions:")
        for i, result in enumerate(results[:10]):
            status = "✓" if result['correct'] else "✗"
            print(f"{status} {result['filename']}: {result['actual_verse_name']} -> {result['predicted_verse_name']} (conf: {result['confidence']:.3f})")
    
    return results

# Training function with history tracking and model saving
def train_model(X, y, save_model=True):
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(np.unique(y_encoded))
    
    # Convert to categorical
    y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)
    
    # Check if we have enough samples for stratified split
    total_samples = len(X)
    test_size = 0.2
    min_test_samples = int(total_samples * test_size)
    
    print(f"Total samples: {total_samples}")
    print(f"Number of classes: {num_classes}")
    print(f"Minimum test samples needed: {min_test_samples}")
    
    # Split data - use stratify only if we have enough samples
    if min_test_samples >= num_classes and total_samples >= num_classes * 2:
        print("Using stratified split...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_categorical, test_size=test_size, random_state=42, stratify=y_encoded
        )
    else:
        print("Dataset too small for stratified split, using random split...")
        # Use smaller test size or random split without stratification
        test_size = max(0.1, min_test_samples / total_samples) if total_samples < 100 else 0.2
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_categorical, test_size=test_size, random_state=42, shuffle=True
        )
    
    # Create model
    model = create_custom_rnn_model((X.shape[1], X.shape[2]), num_classes)
    
    # Custom optimizer
    custom_optimizer = CustomAdam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    
    # Compile model
    model.compile(
        optimizer=custom_optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    print(model.summary())
    
    # Adjust training parameters based on dataset size
    total_train_samples = len(X_train)
    
    # Adjust batch size for small datasets
    if total_train_samples < 100:
        batch_size = min(8, total_train_samples // 4)  # Very small batch for tiny datasets
        epochs = 50  # Fewer epochs for small datasets
        patience = 5
    elif total_train_samples < 500:
        batch_size = min(16, total_train_samples // 8)
        epochs = 75
        patience = 7
    else:
        batch_size = 32
        epochs = 100
        patience = 10
    
    print(f"Training with batch_size={batch_size}, epochs={epochs}, patience={patience}")
    
    # Callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=patience, restore_best_weights=True
    )
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=max(2, patience//2), min_lr=1e-7
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Save model and metadata if requested
    if save_model:
        save_model_and_metadata(model, label_encoder, history)
    
    return model, history, label_encoder

# Plotting function using Plotly
def plot_training_history(history):
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Training & Validation Loss', 'Training & Validation Accuracy',
                       'Training & Validation Precision', 'Training & Validation Recall'),
        vertical_spacing=0.1
    )
    
    epochs = range(1, len(history.history['loss']) + 1)
    
    # Loss plot
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['loss'], 
                  mode='lines', name='Training Loss', line=dict(color='red')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_loss'], 
                  mode='lines', name='Validation Loss', line=dict(color='orange')),
        row=1, col=1
    )
    
    # Accuracy plot
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['accuracy'], 
                  mode='lines', name='Training Accuracy', line=dict(color='blue')),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_accuracy'], 
                  mode='lines', name='Validation Accuracy', line=dict(color='lightblue')),
        row=1, col=2
    )
    
    # Precision plot
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['precision'], 
                  mode='lines', name='Training Precision', line=dict(color='green')),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_precision'], 
                  mode='lines', name='Validation Precision', line=dict(color='lightgreen')),
        row=2, col=1
    )
    
    # Recall plot
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['recall'], 
                  mode='lines', name='Training Recall', line=dict(color='purple')),
        row=2, col=2
    )
    fig.add_trace(
        go.Scatter(x=list(epochs), y=history.history['val_recall'], 
                  mode='lines', name='Validation Recall', line=dict(color='plum')),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title='Training Progress - Quran Verse Detection (An-Naba)',
        height=800,
        showlegend=True
    )
    
    fig.update_xaxes(title_text="Epochs")
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    fig.update_yaxes(title_text="Accuracy", row=1, col=2)
    fig.update_yaxes(title_text="Precision", row=2, col=1)
    fig.update_yaxes(title_text="Recall", row=2, col=2)
    
    fig.show()
    
    return fig

# Main execution - Quran Verse Detection Training
print("=== QURAN VERSE DETECTION - SURAH AN-NABA ===")
print("Loading and preprocessing data...")

# Load data with file tracking information
X, y, file_info = load_data()

print(f"Data shape: {X.shape}")
print(f"Number of verses (classes): {len(np.unique(y))}")
print(f"Verse range: {min(y)} to {max(y)}")

# Print beberapa contoh data untuk verifikasi
print(f"\nSample data verification:")
for i in range(min(5, len(file_info))):
    info = file_info[i]
    verse_name = "Bismillah" if info['verse_label'] == 0 else f"Ayat {info['verse_label']}"
    print(f"  {info['folder']}/{info['filename']} -> {verse_name} (label: {info['verse_label']})")

# Cek apakah dataset cukup untuk training
if len(X) < 10:
    print(f"\nWarning: Dataset terlalu kecil ({len(X)} samples). Minimal 10 samples diperlukan.")
    print("Pastikan folder sample_1 hingga sample_7 tersedia dan berisi file audio yang valid.")
else:
    print(f"\nDataset ready for training: {len(X)} samples from {len(np.unique(y))} verses")

print("\nStarting training...")
model, history, label_encoder = train_model(X, y, save_model=True)

print("\nGenerating training plots...")
training_plot = plot_training_history(history)

print("\nTraining completed!")
print("Model dan hasil training telah selesai. Grafik menunjukkan progress training untuk deteksi ayat Al-Quran Surah An-Naba.")
print("Model, label encoder, dan metadata telah disimpan di folder 'model_saves'")

In [ ]:
# ======= CONTOH PENGGUNAAN MODEL YANG TELAH DISIMPAN =======
# Jalankan code ini untuk menggunakan model yang sudah dilatih sebelumnya

# 1. Load model yang telah disimpan
print("Loading saved model...")
saved_model, saved_label_encoder, metadata, saved_history = load_model_and_metadata("model_saves")

if saved_model is not None:
    print("\n=== Model Information ===")
    print(f"Number of classes: {metadata['num_classes']}")
    print(f"Verse labels: {metadata['verse_labels'][:10]}...")  # Show first 10
    print(f"Input shape: {metadata['input_shape']}")
    print(f"Final training accuracy: {metadata['final_accuracy']:.4f}")
    print(f"Final training loss: {metadata['final_loss']:.4f}")
    
    # 2. Test pada single file
    print("\n=== Testing Single File ===")
    # Contoh test pada file dari sample_1
    test_file = r"d:\new_project\quran_detect\sample_1\078000.mp3"
    if os.path.exists(test_file):
        predicted_verse, confidence = predict_verse(saved_model, saved_label_encoder, test_file)
        print(f"File: {test_file}")
        print(f"Predicted verse: {predicted_verse}")
        print(f"Confidence: {confidence:.4f}")
        
        # Get actual verse from filename
        actual_verse = int(os.path.basename(test_file).split('.')[0][-3:])
        print(f"Actual verse: {actual_verse}")
        print(f"Prediction correct: {predicted_verse == actual_verse}")
    
    # 3. Test pada seluruh folder (gunakan salah satu folder yang belum digunakan untuk training)
    print("\n=== Testing on Folder ===")
    # Misalnya test pada sample_1 (atau buat folder test baru)
    test_folder = r"d:\new_project\quran_detect\sample_1"
    if os.path.exists(test_folder):
        test_results = test_on_folder(saved_model, saved_label_encoder, test_folder)
        
        if test_results:
            # Analisis hasil test
            correct_predictions = [r for r in test_results if r['correct']]
            wrong_predictions = [r for r in test_results if not r['correct']]
            
            print(f"\n=== Detailed Analysis ===")
            if wrong_predictions:
                print("Wrong predictions:")
                for wrong in wrong_predictions[:5]:  # Show first 5 wrong predictions
                    print(f"  {wrong['filename']}: {wrong['actual_verse']} -> {wrong['predicted_verse']} (conf: {wrong['confidence']:.3f})")
            
            # Average confidence
            avg_confidence = np.mean([r['confidence'] for r in test_results])
            print(f"Average confidence: {avg_confidence:.4f}")
            
            # Confidence for correct vs wrong predictions
            if correct_predictions:
                avg_correct_conf = np.mean([r['confidence'] for r in correct_predictions])
                print(f"Average confidence for correct predictions: {avg_correct_conf:.4f}")
            
            if wrong_predictions:
                avg_wrong_conf = np.mean([r['confidence'] for r in wrong_predictions])
                print(f"Average confidence for wrong predictions: {avg_wrong_conf:.4f}")

else:
    print("No saved model found. Please run the training first.")

In [ ]:
# ======= UTILITY FUNCTIONS UNTUK EVALUASI DAN TESTING =======

def evaluate_model_performance(model, label_encoder, test_folders):
    """
    Evaluasi performa model pada multiple test folders
    """
    all_results = {}
    
    for folder_name in test_folders:
        folder_path = os.path.join(r"d:\new_project\quran_detect", folder_name)
        if os.path.exists(folder_path):
            print(f"\nEvaluating on {folder_name}...")
            results = test_on_folder(model, label_encoder, folder_path)
            all_results[folder_name] = results
        else:
            print(f"Folder {folder_path} not found, skipping...")
    
    return all_results

def create_confusion_matrix_data(test_results):
    """
    Buat data untuk confusion matrix dari hasil test
    """
    actual_verses = [r['actual_verse'] for r in test_results]
    predicted_verses = [r['predicted_verse'] for r in test_results]
    
    # Get unique verses
    unique_verses = sorted(list(set(actual_verses + predicted_verses)))
    
    # Create confusion matrix
    confusion_matrix = np.zeros((len(unique_verses), len(unique_verses)))
    
    for actual, predicted in zip(actual_verses, predicted_verses):
        actual_idx = unique_verses.index(actual)
        predicted_idx = unique_verses.index(predicted)
        confusion_matrix[actual_idx][predicted_idx] += 1
    
    return confusion_matrix, unique_verses

def predict_batch_files(model, label_encoder, file_paths):
    """
    Prediksi batch files sekaligus
    """
    results = []
    
    print(f"Processing {len(file_paths)} files...")
    for file_path in tqdm(file_paths):
        if os.path.exists(file_path):
            predicted_verse, confidence = predict_verse(model, label_encoder, file_path)
            results.append({
                'file_path': file_path,
                'filename': os.path.basename(file_path),
                'predicted_verse': predicted_verse,
                'confidence': confidence
            })
        else:
            print(f"File not found: {file_path}")
    
    return results

def save_predictions_to_csv(predictions, output_file="predictions.csv"):
    """
    Simpan hasil prediksi ke CSV file
    """
    df = pd.DataFrame(predictions)
    df.to_csv(output_file, index=False)
    print(f"Predictions saved to: {output_file}")
    return df

# Contoh penggunaan batch prediction
print("=== BATCH PREDICTION EXAMPLE ===")
print("Uncomment the code below to run batch prediction:")

# # Contoh untuk predict multiple files
# file_list = [
#     r"d:\new_project\quran_detect\sample_1\078000.mp3",
#     r"d:\new_project\quran_detect\sample_1\078001.mp3",
#     r"d:\new_project\quran_detect\sample_1\078002.mp3",
#     # Tambahkan file lain...
# ]

# if saved_model is not None:
#     batch_results = predict_batch_files(saved_model, saved_label_encoder, file_list)
#     predictions_df = save_predictions_to_csv(batch_results, "batch_predictions.csv")
#     print(predictions_df.head())

print("\nModel telah siap untuk digunakan!")
print("File yang tersimpan:")
print("- model_saves/quran_verse_model.h5 (Model TensorFlow)")
print("- model_saves/label_encoder.pkl (Label Encoder)")
print("- model_saves/model_metadata.json (Metadata)")
print("- model_saves/training_history.pkl (Training History)")

In [ ]:
# ======= CONTOH TESTING DENGAN PENJELASAN DETAIL =======

def demo_single_prediction(model, label_encoder, audio_file):
    """
    Demo prediksi single file dengan penjelasan detail
    """
    print(f"=== TESTING FILE: {os.path.basename(audio_file)} ===")
    
    if not os.path.exists(audio_file):
        print(f"File not found: {audio_file}")
        return
    
    # Get actual verse from filename
    actual_verse = int(os.path.basename(audio_file).split('.')[0][-3:])
    actual_name = get_verse_name(actual_verse)
    
    print(f"Expected: {actual_name}")
    
    # Predict
    result = predict_verse_with_details(model, label_encoder, audio_file)
    
    if result:
        print(f"Predicted: {result['verse_name']}")
        print(f"Confidence: {result['confidence_percent']:.1f}%")
        
        is_correct = result['verse_number'] == actual_verse
        print(f"Result: {'✓ CORRECT' if is_correct else '✗ WRONG'}")
        
        if result['confidence'] < 0.5:
            print("⚠️  Low confidence - model might be uncertain")
        elif result['confidence'] > 0.9:
            print("✨ High confidence - model is very sure")
    else:
        print("❌ Failed to process audio file")
    
    print("-" * 50)

def demo_batch_testing(model, label_encoder, test_folder):
    """
    Demo testing pada folder dengan analysis yang detail
    """
    print(f"=== BATCH TESTING: {os.path.basename(test_folder)} ===")
    
    if not os.path.exists(test_folder):
        print(f"Folder not found: {test_folder}")
        return
    
    # Test semua files di folder
    results = test_on_folder(model, label_encoder, test_folder)
    
    if results:
        # Analysis per jenis ayat
        print(f"\n=== PERFORMANCE ANALYSIS ===")
        
        # Group by verse type
        bismillah_results = [r for r in results if r['actual_verse'] == 0]
        verse_results = [r for r in results if r['actual_verse'] > 0]
        
        if bismillah_results:
            bismillah_acc = sum([r['correct'] for r in bismillah_results]) / len(bismillah_results)
            print(f"Bismillah detection: {bismillah_acc:.1%} ({len(bismillah_results)} samples)")
        
        if verse_results:
            verse_acc = sum([r['correct'] for r in verse_results]) / len(verse_results)
            print(f"Verse detection: {verse_acc:.1%} ({len(verse_results)} samples)")
        
        # Confidence analysis
        all_confidence = [r['confidence'] for r in results]
        correct_confidence = [r['confidence'] for r in results if r['correct']]
        wrong_confidence = [r['confidence'] for r in results if not r['correct']]
        
        print(f"\nConfidence Analysis:")
        print(f"Average confidence: {np.mean(all_confidence):.3f}")
        if correct_confidence:
            print(f"Correct predictions avg confidence: {np.mean(correct_confidence):.3f}")
        if wrong_confidence:
            print(f"Wrong predictions avg confidence: {np.mean(wrong_confidence):.3f}")
        
        # Show worst predictions
        if wrong_confidence:
            wrong_results = [r for r in results if not r['correct']]
            worst_results = sorted(wrong_results, key=lambda x: x['confidence'], reverse=True)[:3]
            
            print(f"\nMost confident wrong predictions:")
            for r in worst_results:
                print(f"  {r['filename']}: {r['actual_verse_name']} -> {r['predicted_verse_name']} (conf: {r['confidence']:.3f})")

# Jalankan demo jika model sudah di-load
print("=== DEMO FUNCTIONS READY ===")
print("Setelah model selesai training dan disimpan, Anda bisa menggunakan:")
print("1. demo_single_prediction(saved_model, saved_label_encoder, 'path/to/audio.mp3')")
print("2. demo_batch_testing(saved_model, saved_label_encoder, 'path/to/folder')")
print("\nContoh usage:")
print("demo_single_prediction(saved_model, saved_label_encoder, r'd:\\new_project\\quran_detect\\sample_1\\078000.mp3')")
print("demo_batch_testing(saved_model, saved_label_encoder, r'd:\\new_project\\quran_detect\\sample_2')")

In [ ]:
# ======= TEST FILE SPESIFIK: test.mp3 =======

def test_specific_file(model, label_encoder, file_path="test.mp3"):
    """
    Test file audio spesifik (test.mp3) dengan analisis detail
    """
    print("=" * 60)
    print("🎵 TESTING FILE: test.mp3")
    print("=" * 60)
    
    # Cek berbagai lokasi file test.mp3
    possible_paths = [
        file_path,  # Langsung dari parameter
        os.path.join(os.getcwd(), file_path),  # Di working directory
        os.path.join(r"d:\new_project\quran_detect", file_path),  # Di folder project
    ]
    
    test_file_path = None
    for path in possible_paths:
        if os.path.exists(path):
            test_file_path = path
            print(f"✅ File ditemukan: {path}")
            break
    
    if test_file_path is None:
        print("❌ File test.mp3 tidak ditemukan!")
        print("📍 Lokasi yang dicari:")
        for path in possible_paths:
            print(f"   - {path}")
        print("\n💡 Saran:")
        print("1. Pastikan file test.mp3 ada di salah satu lokasi di atas")
        print("2. Atau gunakan path lengkap: test_specific_file(model, label_encoder, r'path\\lengkap\\ke\\test.mp3')")
        return None
    
    print(f"📂 Full path: {test_file_path}")
    
    # Cek info file
    try:
        file_size = os.path.getsize(test_file_path)
        print(f"📊 File size: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
    except:
        print("⚠️  Tidak bisa membaca info file")
    
    print("\n" + "─" * 50)
    print("🔍 MEMPROSES AUDIO...")
    print("─" * 50)
    
    # Prediksi dengan detail
    try:
        result = predict_verse_with_details(model, label_encoder, test_file_path)
        
        if result is None:
            print("❌ Gagal memproses file audio!")
            print("💡 Kemungkinan penyebab:")
            print("   - Format file tidak didukung")
            print("   - File audio rusak")
            print("   - Audio terlalu pendek atau terlalu panjang")
            return None
        
        print("✅ Audio berhasil diproses!")
        print("\n" + "=" * 50)
        print("🎯 HASIL PREDIKSI")
        print("=" * 50)
        
        # Tampilkan hasil prediksi
        print(f"📝 Prediksi: {result['verse_name']}")
        print(f"🔢 Label ID: {result['verse_number']}")
        print(f"📊 Confidence: {result['confidence_percent']:.1f}%")
        
        # Interpretasi confidence
        confidence = result['confidence']
        if confidence >= 0.9:
            confidence_level = "🟢 SANGAT TINGGI"
            interpretation = "Model sangat yakin dengan prediksinya"
        elif confidence >= 0.7:
            confidence_level = "🟡 TINGGI"
            interpretation = "Model cukup yakin dengan prediksinya"
        elif confidence >= 0.5:
            confidence_level = "🟠 SEDANG"
            interpretation = "Model agak ragu, perlu verifikasi manual"
        else:
            confidence_level = "🔴 RENDAH"
            interpretation = "Model tidak yakin, kemungkinan prediksi salah"
        
        print(f"📈 Tingkat Keyakinan: {confidence_level}")
        print(f"💭 Interpretasi: {interpretation}")
        
        # Detail tambahan
        print(f"\n📋 Detail Teknis:")
        print(f"   - File: {result['filename']}")
        print(f"   - Raw confidence: {confidence:.6f}")
        
        # Saran tindak lanjut
        print(f"\n💡 Saran:")
        if confidence >= 0.8:
            print("   ✅ Prediksi dapat dipercaya")
        elif confidence >= 0.6:
            print("   ⚠️  Disarankan untuk cross-check manual")
        else:
            print("   ❌ Prediksi mungkin tidak akurat, coba file audio yang lebih jelas")
        
        return result
        
    except Exception as e:
        print(f"❌ Error saat memproses: {str(e)}")
        print("💡 Coba restart kernel atau check file audio")
        return None

# ======= JALANKAN TEST untuk test.mp3 =======

print("🚀 Menjalankan test untuk file test.mp3...")
print("⏳ Pastikan file test.mp3 sudah tersedia di folder project")

# Test menggunakan model yang sudah diload
if 'saved_model' in locals() and 'saved_label_encoder' in locals():
    result = test_specific_file(saved_model, saved_label_encoder, "test.mp3")
    
    if result:
        print(f"\n🎉 SUCCESS! File test.mp3 diprediksi sebagai: {result['verse_name']}")
    else:
        print(f"\n❌ Gagal memproses test.mp3. Pastikan file tersedia dan valid.")
else:
    print("❌ Model belum di-load. Jalankan cell training terlebih dahulu!")
    print("💡 Atau load model dengan: model, label_encoder, _, _ = load_model_and_metadata()")

In [ ]:
# ======= CUSTOM FILE TESTING =======
# Gunakan cell ini untuk test file dengan nama atau path yang berbeda

def quick_test_audio(file_path):
    """
    Quick test untuk file audio apapun
    """
    if 'saved_model' not in locals() or 'saved_label_encoder' not in locals():
        print("❌ Model belum di-load!")
        return
    
    print(f"🎵 Testing: {os.path.basename(file_path)}")
    
    if not os.path.exists(file_path):
        print(f"❌ File tidak ditemukan: {file_path}")
        return
    
    result = predict_verse_with_details(saved_model, saved_label_encoder, file_path)
    
    if result:
        print(f"📝 Hasil: {result['verse_name']}")
        print(f"📊 Confidence: {result['confidence_percent']:.1f}%")
        
        if result['confidence'] >= 0.8:
            print("✅ Prediksi sangat yakin")
        elif result['confidence'] >= 0.6:
            print("⚠️  Prediksi cukup yakin")
        else:
            print("❌ Prediksi kurang yakin")
    else:
        print("❌ Gagal memproses file")


test_specific_file(saved_model, saved_label_encoder, 'test.mp3')

# ======= CONTOH PENGGUNAAN =======
print("📝 Contoh penggunaan:")
print("quick_test_audio('test.mp3')")
print("quick_test_audio(r'd:\\path\\to\\your\\audio.mp3')")
print("test_specific_file(saved_model, saved_label_encoder, 'your_file.mp3')")

print("\n" + "="*50)
print("💡 INSTRUKSI PENGGUNAAN:")
print("1. Letakkan file test.mp3 di folder project ini")
print("2. Atau gunakan path lengkap ke file audio Anda")
print("3. Jalankan: quick_test_audio('nama_file.mp3')")
print("="*50)

In [ ]:
# ======= IMPROVED AUDIO PREPROCESSING & MODEL =======

import scipy.signal
from sklearn.preprocessing import StandardScaler, RobustScaler
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import regularizers
import scipy.stats

def extract_advanced_features(file_path, max_length=256, sr=22050):
    """
    Extract features audio yang lebih comprehensive untuk deteksi yang lebih baik
    """
    try:
        # Load audio dengan preprocessing
        audio, sample_rate = librosa.load(file_path, sr=sr)
        
        # 1. Audio Preprocessing
        # Remove silence dan normalize
        audio = librosa.util.normalize(audio)
        
        # Trim silence dari awal dan akhir
        audio, _ = librosa.effects.trim(audio, top_db=20)
        
        # Apply pre-emphasis filter
        audio = scipy.signal.lfilter([1, -0.95], [1], audio)
        
        # 2. Feature Extraction yang lebih lengkap
        features_list = []
        
        # MFCC dengan konfigurasi yang lebih baik
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20, n_fft=2048, hop_length=512)
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)
        
        features_list.extend([mfccs, mfcc_delta, mfcc_delta2])
        
        # Spectral features
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr, roll_percent=0.85)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr, n_bands=6)
        
        features_list.extend([spectral_centroids, spectral_rolloff, spectral_bandwidth, spectral_contrast])
        
        # Rhythm features
        zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)
        tempo, beats = librosa.beat.beat_track(y=audio, sr=sr)
        
        # Chroma features (untuk pitch information)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        
        # Tonnetz (harmonic features)
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
        
        features_list.extend([zero_crossing_rate, chroma, tonnetz])
        
        # 3. Combine all features
        combined_features = np.vstack(features_list)
        
        # 4. Feature normalization dan padding/truncation
        # Normalize each feature type
        scaler = RobustScaler()
        combined_features = scaler.fit_transform(combined_features.T).T
        
        # Pad or truncate to fixed length
        if combined_features.shape[1] < max_length:
            pad_width = max_length - combined_features.shape[1]
            combined_features = np.pad(combined_features, ((0, 0), (0, pad_width)), mode='constant')
        else:
            combined_features = combined_features[:, :max_length]
        
        # Transpose untuk format yang diharapkan model (time_steps, features)
        return combined_features.T
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def create_improved_model(input_shape, num_classes):
    """
    Model yang diperbaiki dengan arsitektur yang lebih advanced
    """
    inputs = tf.keras.Input(shape=input_shape)
    
    # 1. Convolutional layers untuk pattern extraction
    x = layers.Reshape((input_shape[0], input_shape[1], 1))(inputs)
    
    # Conv1D layers untuk temporal pattern
    x = layers.Conv1D(64, kernel_size=5, activation='relu', padding='same')(layers.Reshape((input_shape[0], input_shape[1]))(inputs))
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    
    # 2. Attention mechanism
    attention = layers.Dense(128, activation='tanh')(x)
    attention = layers.Dense(1, activation='softmax')(attention)
    x = layers.Multiply()([x, attention])
    
    # 3. Bidirectional LSTM layers
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.3))(x)
    
    # 4. Dense layers dengan regularization
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Custom learning rate scheduler (Fixed)
class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, warmup_steps, total_steps):
        super(WarmupCosineDecay, self).__init__()
        self.initial_learning_rate = tf.cast(initial_learning_rate, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)
        self.total_steps = tf.cast(total_steps, tf.float32)
    
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        
        # Warmup phase
        warmup_lr = (self.initial_learning_rate * step / self.warmup_steps)
        
        # Cosine decay phase
        cosine_lr = self.initial_learning_rate * 0.5 * (1 + tf.cos(
            tf.constant(3.14159265359, dtype=tf.float32) * (step - self.warmup_steps) / (self.total_steps - self.warmup_steps)
        ))
        
        return tf.where(step < self.warmup_steps, warmup_lr, cosine_lr)

def load_data_improved():
    """
    Load data dengan improved preprocessing
    """
    base_path = r"d:\new_project\quran_detect"
    
    # Cek semua folder sample
    available_folders = []
    for i in range(1, 8):
        folder_name = f'sample_{i}'
        folder_path = os.path.join(base_path, folder_name)
        if os.path.exists(folder_path):
            available_folders.append(folder_name)
    
    print(f"Found {len(available_folders)} sample folders: {available_folders}")
    print("Using improved feature extraction...")
    
    X = []
    y = []
    file_info = []
    
    for folder in available_folders:
        folder_path = os.path.join(base_path, folder)
        print(f"Processing {folder} with advanced features...")
        
        files = sorted([f for f in os.listdir(folder_path) if f.endswith('.mp3')])
        
        for file in tqdm(files, desc=f"Processing {folder}"):
            file_path = os.path.join(folder_path, file)
            
            # Use improved feature extraction
            features = extract_advanced_features(file_path, max_length=256)
            
            if features is not None:
                X.append(features)
                verse_num = int(file.split('.')[0][-3:])
                y.append(verse_num)
                
                file_info.append({
                    'folder': folder,
                    'filename': file,
                    'verse_label': verse_num
                })
            else:
                print(f"  Failed to extract features from: {file}")
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"\n=== Improved Dataset Summary ===")
    print(f"Total samples: {len(X)}")
    print(f"Feature shape per sample: {X[0].shape if len(X) > 0 else 'N/A'}")
    print(f"Feature dimensions: {X.shape[2] if len(X) > 0 else 'N/A'} features per timestep")
    print(f"Verse range: {min(y)} to {max(y)}")
    print(f"Unique verses: {len(np.unique(y))}")
    
    return X, y, file_info

print("✅ Improved preprocessing and model functions loaded!")
print("📈 Key improvements:")
print("   - Advanced audio preprocessing (noise reduction, normalization)")
print("   - Comprehensive feature extraction (MFCC + deltas, spectral, chroma, etc.)")
print("   - Improved model architecture (Conv1D + Attention + BiLSTM)")
print("   - Better optimizer with learning rate scheduling")
print("   - Regularization untuk prevent overfitting")

In [ ]:
# ======= IMPROVED TRAINING FUNCTION =======

def train_improved_model(X, y, save_model=True, model_name="improved_quran_model"):
    """
    Training function dengan berbagai perbaikan untuk akurasi yang lebih tinggi
    """
    print("🚀 Starting improved training...")
    
    # 1. Label encoding
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(np.unique(y_encoded))
    
    # Convert to categorical
    y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)
    
    # 2. Data Augmentation untuk training set
    def augment_audio_features(features, noise_factor=0.02):
        """Simple data augmentation"""
        # Add small random noise
        noise = np.random.normal(0, noise_factor, features.shape)
        return features + noise
    
    # 3. Stratified split dengan data augmentation
    from sklearn.model_selection import StratifiedKFold
    
    # Use more sophisticated split
    test_size = 0.15  # Smaller test set, larger training set
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=test_size, random_state=42, stratify=y_encoded
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    
    # 4. Data augmentation for training set
    print("Applying data augmentation...")
    X_train_aug = []
    y_train_aug = []
    
    for i in range(len(X_train)):
        # Original sample
        X_train_aug.append(X_train[i])
        y_train_aug.append(y_train[i])
        
        # Augmented samples (2 variations per original)
        aug1 = augment_audio_features(X_train[i], noise_factor=0.01)
        aug2 = augment_audio_features(X_train[i], noise_factor=0.02)
        
        X_train_aug.extend([aug1, aug2])
        y_train_aug.extend([y_train[i], y_train[i]])
    
    X_train_aug = np.array(X_train_aug)
    y_train_aug = np.array(y_train_aug)
    
    print(f"After augmentation: {len(X_train_aug)} training samples")
    
    # 5. Create improved model
    model = create_improved_model((X.shape[1], X.shape[2]), num_classes)
    
    # 6. Advanced optimizer dengan learning rate scheduling
    total_steps = (len(X_train_aug) // 16) * 100  # batch_size=16, epochs=100
    warmup_steps = total_steps // 10
    
    lr_schedule = WarmupCosineDecay(
        initial_learning_rate=0.001,
        warmup_steps=warmup_steps,
        total_steps=total_steps
    )
    
    # Use AdamW optimizer (better than standard Adam)
    optimizer = AdamW(
        learning_rate=lr_schedule,
        weight_decay=0.01,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8
    )
    
    # 7. Compile model dengan custom metrics
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy'), 
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.F1Score(name='f1_score')]
    )
    
    print("📊 Model Architecture:")
    model.summary()
    
    # 8. Advanced callbacks
    model_dir = f"model_saves_{model_name}"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    callbacks = [
        # Early stopping with patience
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Model checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(model_dir, 'best_model.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        
        # Reduce learning rate on plateau
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.7,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        
        # CSV logger
        tf.keras.callbacks.CSVLogger(
            os.path.join(model_dir, 'training_log.csv')
        )
    ]
    
    # 9. Determine training parameters
    total_samples = len(X_train_aug)
    if total_samples < 500:
        batch_size = 8
        epochs = 80
    else:
        batch_size = 16
        epochs = 100
    
    print(f"Training parameters:")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Epochs: {epochs}")
    print(f"  - Learning rate: Adaptive with warmup")
    print(f"  - Data augmentation: Applied")
    
    # 10. Train model
    print(f"🎯 Starting training...")
    history = model.fit(
        X_train_aug, y_train_aug,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )
    
    # 11. Evaluate model
    print(f"\n📈 Final Evaluation:")
    train_loss, train_acc, train_top3, train_prec, train_rec, train_f1 = model.evaluate(X_train_aug, y_train_aug, verbose=0)
    test_loss, test_acc, test_top3, test_prec, test_rec, test_f1 = model.evaluate(X_test, y_test, verbose=0)
    
    print(f"Training - Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_rec:.4f}, F1: {train_f1:.4f}")
    print(f"Testing  - Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}")
    print(f"Top-3 Accuracy: {test_top3:.4f}")
    
    # 12. Save model and metadata
    if save_model:
        save_improved_model_metadata(model, label_encoder, history, model_dir, {
            'train_accuracy': train_acc,
            'test_accuracy': test_acc,
            'train_f1': train_f1,
            'test_f1': test_f1,
            'top3_accuracy': test_top3,
            'data_augmentation': True,
            'feature_extraction': 'advanced',
            'model_type': 'improved_cnn_bilstm_attention'
        })
    
    return model, history, label_encoder

def save_improved_model_metadata(model, label_encoder, history, model_dir, extra_metrics):
    """
    Save improved model with enhanced metadata
    """
    # Save model
    model_path = os.path.join(model_dir, "improved_quran_model.h5")
    model.save(model_path)
    print(f"✅ Model saved to: {model_path}")
    
    # Save label encoder
    encoder_path = os.path.join(model_dir, "label_encoder.pkl")
    with open(encoder_path, 'wb') as f:
        pickle.dump(label_encoder, f)
    print(f"✅ Label encoder saved to: {encoder_path}")
    
    # Save training history
    history_path = os.path.join(model_dir, "training_history.pkl")
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    print(f"✅ Training history saved to: {history_path}")
    
    # Enhanced metadata
    metadata = {
        "model_version": "improved_v2",
        "num_classes": len(label_encoder.classes_),
        "verse_labels": label_encoder.classes_.tolist(),
        "input_shape": model.input_shape[1:],
        "total_epochs": len(history.history['loss']),
        "best_val_accuracy": max(history.history['val_accuracy']),
        "best_val_loss": min(history.history['val_loss']),
        "final_metrics": extra_metrics,
        "improvements": [
            "Advanced audio preprocessing",
            "Comprehensive feature extraction",
            "CNN + BiLSTM + Attention architecture", 
            "AdamW optimizer with learning rate scheduling",
            "Data augmentation",
            "Regularization techniques"
        ]
    }
    
    metadata_path = os.path.join(model_dir, "model_metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Enhanced metadata saved to: {metadata_path}")

print("🎯 Improved training function ready!")
print("📋 Key improvements in training:")
print("   ✅ Data augmentation untuk increase dataset size")
print("   ✅ AdamW optimizer dengan learning rate scheduling")
print("   ✅ Advanced callbacks dan monitoring")
print("   ✅ Comprehensive evaluation metrics")
print("   ✅ Better train/test split strategy")

In [ ]:
# ======= ALTERNATIVE SIMPLIFIED TRAINING (JIKA ADA ERROR) =======

def train_improved_model_simple(X, y, save_model=True, model_name="improved_simple"):
    """
    Training function yang disederhanakan jika ada masalah dengan advanced scheduler
    """
    print("🚀 Starting simplified improved training...")
    
    # 1. Label encoding
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(np.unique(y_encoded))
    
    # Convert to categorical
    y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)
    
    # 2. Data Augmentation
    def augment_audio_features(features, noise_factor=0.02):
        noise = np.random.normal(0, noise_factor, features.shape)
        return features + noise
    
    # 3. Train-test split
    test_size = 0.15
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=test_size, random_state=42, stratify=y_encoded
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    
    # 4. Data augmentation
    print("Applying data augmentation...")
    X_train_aug = []
    y_train_aug = []
    
    for i in range(len(X_train)):
        # Original sample
        X_train_aug.append(X_train[i])
        y_train_aug.append(y_train[i])
        
        # Augmented samples
        aug1 = augment_audio_features(X_train[i], noise_factor=0.01)
        aug2 = augment_audio_features(X_train[i], noise_factor=0.02)
        
        X_train_aug.extend([aug1, aug2])
        y_train_aug.extend([y_train[i], y_train[i]])
    
    X_train_aug = np.array(X_train_aug)
    y_train_aug = np.array(y_train_aug)
    
    print(f"After augmentation: {len(X_train_aug)} training samples")
    
    # 5. Create improved model
    model = create_improved_model((X.shape[1], X.shape[2]), num_classes)
    
    # 6. Simplified optimizer (no custom scheduler)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8
    )
    
    # 7. Compile model
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy'), 
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
    
    print("📊 Model Architecture:")
    model.summary()
    
    # 8. Simplified callbacks
    model_dir = f"model_saves_{model_name}"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(model_dir, 'best_model.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.7,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    # 9. Training parameters
    total_samples = len(X_train_aug)
    if total_samples < 500:
        batch_size = 8
        epochs = 80
    else:
        batch_size = 16
        epochs = 100
    
    print(f"Training parameters:")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Epochs: {epochs}")
    print(f"  - Optimizer: Simplified Adam")
    
    # 10. Train model
    print(f"🎯 Starting training...")
    history = model.fit(
        X_train_aug, y_train_aug,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )
    
    # 11. Evaluate model
    print(f"\n📈 Final Evaluation:")
    try:
        train_metrics = model.evaluate(X_train_aug, y_train_aug, verbose=0)
        test_metrics = model.evaluate(X_test, y_test, verbose=0)
        
        print(f"Training - Accuracy: {train_metrics[1]:.4f}, Precision: {train_metrics[3]:.4f}, Recall: {train_metrics[4]:.4f}")
        print(f"Testing  - Accuracy: {test_metrics[1]:.4f}, Precision: {test_metrics[3]:.4f}, Recall: {test_metrics[4]:.4f}")
        print(f"Top-3 Accuracy: {test_metrics[2]:.4f}")
        
        extra_metrics = {
            'train_accuracy': train_metrics[1],
            'test_accuracy': test_metrics[1],
            'test_precision': test_metrics[3],
            'test_recall': test_metrics[4],
            'top3_accuracy': test_metrics[2],
            'data_augmentation': True,
            'feature_extraction': 'advanced',
            'model_type': 'simplified_improved'
        }
    except Exception as e:
        print(f"Evaluation error: {e}")
        extra_metrics = {
            'train_accuracy': 0,
            'test_accuracy': 0,
            'model_type': 'simplified_improved'
        }
    
    # 12. Save model
    if save_model:
        save_improved_model_metadata(model, label_encoder, history, model_dir, extra_metrics)
    
    return model, history, label_encoder

print("🔧 Simplified training function ready!")
print("💡 Use this if the advanced training function has issues with TensorFlow versions")

In [ ]:
# ======= RESTART TRAINING DENGAN ERROR HANDLING =======

print("🔄 RESTARTING IMPROVED TRAINING")
print("=" * 50)

# Check apakah data sudah di-load
if 'X_improved' in locals() and 'y_improved' in locals():
    print(f"✅ Data sudah tersedia: {len(X_improved)} samples")
    
    # Try training dengan error handling
    try:
        print("🚀 Attempting simplified training (more stable)...")
        restart_model, restart_history, restart_label_encoder = train_improved_model_simple(
            X_improved, y_improved, save_model=True, model_name="improved_restart"
        )
        
        print(f"\n🎉 TRAINING BERHASIL!")
        print("=" * 40)
        
        # Plot hasil training
        print("📊 Generating training plots...")
        restart_plot = plot_training_history(restart_history)
        
        print(f"\n📁 Model disimpan di: model_saves_improved_restart/")
        print(f"🎯 Model siap untuk testing!")
        
        # Quick evaluation
        try:
            final_metrics = restart_model.evaluate(X_improved[:50], 
                tf.keras.utils.to_categorical(LabelEncoder().fit_transform(y_improved[:50]), 
                len(np.unique(y_improved))), verbose=0)
            print(f"📊 Quick test accuracy: {final_metrics[1]:.3f}")
        except:
            print("📊 Quick test skipped")
            
    except Exception as e:
        print(f"❌ Training gagal: {e}")
        print("\n💡 Troubleshooting:")
        print("1. Restart kernel jika perlu")
        print("2. Check memory availability")  
        print("3. Reduce batch size atau epochs")
        print("4. Verify TensorFlow installation")
        
else:
    print("❌ Data belum di-load!")
    print("💡 Jalankan cell load data terlebih dahulu:")
    print("   X_improved, y_improved, file_info_improved = load_data_improved()")

In [ ]:
# ======= ULTRA SIMPLE TRAINING (GUARANTEED TO WORK) =======

def train_basic_improved_model(X, y, save_model=True, model_name="basic_improved"):
    """
    Training function yang sangat basic tapi menggunakan improved features
    Guaranteed to work pada semua versi TensorFlow
    """
    print("🔧 Starting ultra-simple but improved training...")
    
    # 1. Basic label encoding
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(np.unique(y_encoded))
    y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)
    
    # 2. Simple train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    print(f"Feature shape: {X_train.shape}")
    
    # 3. Create basic improved model (simplified architecture)
    inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2]))
    
    # Simple but effective architecture
    x = layers.LSTM(64, return_sequences=True, dropout=0.3)(inputs)
    x = layers.LSTM(32, return_sequences=False, dropout=0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=x)
    
    # 4. Basic optimizer and compilation
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("📊 Basic Model Architecture:")
    model.summary()
    
    # 5. Simple callbacks
    model_dir = f"model_saves_{model_name}"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5
        )
    ]
    
    # 6. Train model
    print("🎯 Starting basic training...")
    history = model.fit(
        X_train, y_train,
        batch_size=16,
        epochs=50,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1
    )
    
    # 7. Evaluate
    train_acc = model.evaluate(X_train, y_train, verbose=0)[1]
    test_acc = model.evaluate(X_test, y_test, verbose=0)[1]
    
    print(f"\n📈 Results:")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # 8. Save model
    if save_model:
        model_path = os.path.join(model_dir, "basic_improved_model.h5")
        model.save(model_path)
        
        encoder_path = os.path.join(model_dir, "label_encoder.pkl")
        with open(encoder_path, 'wb') as f:
            pickle.dump(label_encoder, f)
        
        metadata = {
            "model_type": "basic_improved",
            "train_accuracy": float(train_acc),
            "test_accuracy": float(test_acc),
            "num_classes": num_classes,
            "input_shape": list(X.shape[1:]),
            "epochs_trained": len(history.history['loss'])
        }
        
        metadata_path = os.path.join(model_dir, "metadata.json")
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ Model saved to: {model_dir}/")
    
    return model, history, label_encoder

# ======= EXECUTE ULTRA SIMPLE TRAINING =======

print("🚀 STARTING GUARANTEED TRAINING")
print("=" * 50)

if 'X_improved' in locals() and 'y_improved' in locals():
    print(f"✅ Using improved features: {X_improved.shape}")
    
    try:
        # Use ultra-simple training that's guaranteed to work
        basic_model, basic_history, basic_encoder = train_basic_improved_model(
            X_improved, y_improved, save_model=True, model_name="basic_improved"
        )
        
        print(f"\n🎉 TRAINING SUCCESSFUL!")
        print("=" * 40)
        
        # Plot results
        print("📊 Generating plots...")
        basic_plot = plot_training_history(basic_history)
        
        print(f"\n✅ Success! Model info:")
        print(f"   - Architecture: Simple LSTM with improved features")
        print(f"   - Features: {X_improved.shape[2]} advanced audio features")
        print(f"   - Classes: {len(np.unique(y_improved))} verses")
        print(f"   - Saved to: model_saves_basic_improved/")
        
        # Test dengan sample data
        try:
            test_sample = X_improved[0:1]
            test_pred = basic_model.predict(test_sample, verbose=0)
            predicted_class = np.argmax(test_pred)
            confidence = np.max(test_pred)
            verse_number = basic_encoder.inverse_transform([predicted_class])[0]
            
            print(f"\n🧪 Quick test:")
            print(f"   Sample prediction: {get_verse_name(verse_number)}")
            print(f"   Confidence: {confidence:.3f}")
        except:
            print(f"\n🧪 Quick test skipped")
            
    except Exception as e:
        print(f"❌ Even basic training failed: {e}")
        print("💡 Try:")
        print("1. Restart kernel")
        print("2. Check TensorFlow installation")
        print("3. Verify data integrity")
        
else:
    print("❌ Improved data not loaded!")
    print("💡 Run: X_improved, y_improved, _ = load_data_improved()")

In [ ]:
# ======= TESTING BASIC IMPROVED MODEL =======

def load_basic_model(model_dir="model_saves_basic_improved"):
    """Load basic improved model"""
    try:
        model_path = os.path.join(model_dir, "basic_improved_model.h5")
        model = tf.keras.models.load_model(model_path)
        
        encoder_path = os.path.join(model_dir, "label_encoder.pkl")
        with open(encoder_path, 'rb') as f:
            label_encoder = pickle.load(f)
        
        metadata_path = os.path.join(model_dir, "metadata.json")
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        print(f"✅ Basic model loaded successfully!")
        print(f"   - Test accuracy: {metadata['test_accuracy']:.3f}")
        print(f"   - Classes: {metadata['num_classes']}")
        
        return model, label_encoder, metadata
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        return None, None, None

def predict_with_basic_model(model, label_encoder, audio_file_path):
    """Predict using basic model dengan improved features"""
    # Extract improved features
    features = extract_advanced_features(audio_file_path)
    if features is None:
        return None, None
    
    # Reshape dan predict
    features = features.reshape(1, features.shape[0], features.shape[1])
    prediction = model.predict(features, verbose=0)
    
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    verse_number = label_encoder.inverse_transform([predicted_class])[0]
    
    return verse_number, confidence

def test_basic_model_performance(model, label_encoder, test_folder, max_files=20):
    """Test basic model performance"""
    if not os.path.exists(test_folder):
        print(f"❌ Test folder not found: {test_folder}")
        return
    
    files = sorted([f for f in os.listdir(test_folder) if f.endswith('.mp3')])[:max_files]
    
    print(f"🧪 Testing {len(files)} files...")
    
    correct = 0
    total = 0
    results = []
    
    for file in tqdm(files, desc="Testing"):
        file_path = os.path.join(test_folder, file)
        actual_verse = int(file.split('.')[0][-3:])
        
        try:
            predicted_verse, confidence = predict_with_basic_model(model, label_encoder, file_path)
            if predicted_verse is not None:
                is_correct = (predicted_verse == actual_verse)
                if is_correct:
                    correct += 1
                total += 1
                
                results.append({
                    'file': file,
                    'actual': actual_verse,
                    'predicted': predicted_verse,
                    'confidence': confidence,
                    'correct': is_correct
                })
        except Exception as e:
            print(f"Error processing {file}: {e}")
    
    if total > 0:
        accuracy = correct / total
        avg_confidence = np.mean([r['confidence'] for r in results])
        
        print(f"\n📊 BASIC MODEL PERFORMANCE:")
        print(f"   Accuracy: {accuracy:.1%} ({correct}/{total})")
        print(f"   Average confidence: {avg_confidence:.3f}")
        
        # Show some examples
        print(f"\n📋 Sample results:")
        for result in results[:5]:
            status = "✅" if result['correct'] else "❌"
            actual_name = get_verse_name(result['actual'])
            pred_name = get_verse_name(result['predicted'])
            print(f"   {status} {result['file']}: {actual_name} -> {pred_name} ({result['confidence']:.2f})")
        
        return results
    else:
        print("❌ No valid results")
        return []

# ======= TEST BASIC MODEL =======

print("🧪 TESTING BASIC IMPROVED MODEL")
print("=" * 50)

# Load basic model jika sudah di-train
if 'basic_model' in locals() and 'basic_encoder' in locals():
    print("✅ Using model from current session")
    test_model = basic_model
    test_encoder = basic_encoder
else:
    print("🔄 Loading saved model...")
    test_model, test_encoder, test_metadata = load_basic_model()

if test_model is not None:
    # Test dengan file test.mp3 jika ada
    test_files = ["test.mp3", "sample_1/078001.mp3", "sample_2/078005.mp3"]
    
    for test_file in test_files:
        full_path = test_file if os.path.exists(test_file) else os.path.join(r"d:\new_project\quran_detect", test_file)
        
        if os.path.exists(full_path):
            print(f"\n🎵 Testing: {os.path.basename(full_path)}")
            try:
                pred_verse, conf = predict_with_basic_model(test_model, test_encoder, full_path)
                if pred_verse is not None:
                    verse_name = get_verse_name(pred_verse)
                    print(f"   Prediction: {verse_name}")
                    print(f"   Confidence: {conf:.3f}")
                    
                    if conf > 0.7:
                        print("   ✅ High confidence")
                    elif conf > 0.5:
                        print("   ⚠️  Medium confidence")
                    else:
                        print("   ❌ Low confidence")
                else:
                    print("   ❌ Failed to process")
            except Exception as e:
                print(f"   Error: {e}")
            break
    
    # Test pada folder sample
    sample_folder = r"d:\new_project\quran_detect\sample_1"
    if os.path.exists(sample_folder):
        print(f"\n🔬 Comprehensive testing on sample folder...")
        test_results = test_basic_model_performance(test_model, test_encoder, sample_folder, max_files=10)
    
else:
    print("❌ No model available for testing")
    print("💡 Train model first using the cell above")

In [ ]:
# ======= JALANKAN TRAINING DENGAN MODEL YANG DIPERBAIKI =======

print("🚀 MEMULAI TRAINING DENGAN MODEL YANG DIPERBAIKI")
print("=" * 60)

# Load data dengan preprocessing yang diperbaiki
print("📥 Loading data dengan improved preprocessing...")
X_improved, y_improved, file_info_improved = load_data_improved()

if len(X_improved) > 0:
    print(f"✅ Data berhasil diload: {len(X_improved)} samples")
    print(f"📊 Feature shape: {X_improved.shape}")
    
    # Bandingkan dengan data sebelumnya
    if 'X' in locals():
        print(f"📈 Improvement comparison:")
        print(f"   - Previous features per timestep: {X.shape[2] if len(X.shape) > 2 else 'N/A'}")
        print(f"   - Improved features per timestep: {X_improved.shape[2]}")
        print(f"   - Previous sequence length: {X.shape[1] if len(X.shape) > 1 else 'N/A'}")
        print(f"   - Improved sequence length: {X_improved.shape[1]}")
    
    print(f"\n🎯 Starting improved training...")
    print("⏳ This may take longer due to:")
    print("   - Advanced feature extraction")
    print("   - Data augmentation") 
    print("   - More complex model architecture")
    
    # Train dengan model yang diperbaiki - try advanced first, fallback to simple
    try:
        print("🚀 Attempting advanced training with custom scheduler...")
        improved_model, improved_history, improved_label_encoder = train_improved_model(
            X_improved, y_improved, save_model=True, model_name="improved_v2"
        )
        print("✅ Advanced training successful!")
    except Exception as e:
        print(f"⚠️  Advanced training failed: {e}")
        print("🔄 Falling back to simplified training...")
        improved_model, improved_history, improved_label_encoder = train_improved_model_simple(
            X_improved, y_improved, save_model=True, model_name="improved_simple"
        )
        print("✅ Simplified training successful!")
    
    print(f"\n🎉 TRAINING COMPLETED!")
    print("=" * 50)
    
    # Plot training history
    print("📊 Generating improved training plots...")
    improved_plot = plot_training_history(improved_history)
    
    print(f"\n✅ Model yang diperbaiki telah selesai!")
    print(f"📁 Files disimpan di: model_saves_improved_v2/")
    print(f"🎯 Sekarang coba test dengan model yang diperbaiki!")
    
else:
    print("❌ Gagal load data. Pastikan folder sample tersedia dan valid.")

In [ ]:
# ======= TESTING MODEL YANG DIPERBAIKI =======

def load_improved_model(model_dir="model_saves_improved_v2"):
    """
    Load improved model yang telah disimpan
    """
    try:
        model_path = os.path.join(model_dir, "improved_quran_model.h5")
        
        # Custom objects untuk load model
        custom_objects = {
            'WarmupCosineDecay': WarmupCosineDecay,
            'AdamW': AdamW
        }
        
        model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
        print(f"✅ Improved model loaded from: {model_path}")
        
        # Load label encoder
        encoder_path = os.path.join(model_dir, "label_encoder.pkl")
        with open(encoder_path, 'rb') as f:
            label_encoder = pickle.load(f)
        print(f"✅ Label encoder loaded")
        
        # Load metadata
        metadata_path = os.path.join(model_dir, "model_metadata.json")
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        print(f"✅ Metadata loaded")
        
        return model, label_encoder, metadata
        
    except Exception as e:
        print(f"❌ Error loading improved model: {e}")
        return None, None, None

def predict_with_improved_model(model, label_encoder, audio_file_path):
    """
    Prediksi menggunakan improved model dengan preprocessing yang sama
    """
    # Extract features dengan improved preprocessing
    features = extract_advanced_features(audio_file_path)
    if features is None:
        return None, None
    
    # Reshape untuk prediksi
    features = features.reshape(1, features.shape[0], features.shape[1])
    
    # Make prediction
    prediction = model.predict(features, verbose=0)
    predicted_class = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    # Get top 3 predictions
    top3_indices = np.argsort(prediction[0])[-3:][::-1]
    top3_probs = prediction[0][top3_indices]
    
    # Convert back to original labels
    verse_number = label_encoder.inverse_transform([predicted_class])[0]
    top3_verses = label_encoder.inverse_transform(top3_indices)
    
    return {
        'verse_number': verse_number,
        'confidence': confidence,
        'top3_predictions': list(zip(top3_verses, top3_probs))
    }

def compare_models_performance(original_model, improved_model, original_encoder, improved_encoder, test_folder):
    """
    Bandingkan performa model original vs improved
    """
    print("🔍 PERBANDINGAN PERFORMA MODEL")
    print("=" * 50)
    
    if not os.path.exists(test_folder):
        print(f"❌ Test folder tidak ditemukan: {test_folder}")
        return
    
    files = sorted([f for f in os.listdir(test_folder) if f.endswith('.mp3')])[:10]  # Test 10 files pertama
    
    results_comparison = []
    
    for file in tqdm(files, desc="Comparing models"):
        file_path = os.path.join(test_folder, file)
        actual_verse = int(file.split('.')[0][-3:])
        
        # Original model prediction
        try:
            orig_verse, orig_conf = predict_verse(original_model, original_encoder, file_path)
            orig_correct = (orig_verse == actual_verse) if orig_verse is not None else False
        except:
            orig_verse, orig_conf, orig_correct = None, 0, False
        
        # Improved model prediction
        try:
            improved_result = predict_with_improved_model(improved_model, improved_encoder, file_path)
            if improved_result:
                imp_verse = improved_result['verse_number']
                imp_conf = improved_result['confidence']
                imp_correct = (imp_verse == actual_verse)
                imp_top3 = improved_result['top3_predictions']
            else:
                imp_verse, imp_conf, imp_correct, imp_top3 = None, 0, False, []
        except:
            imp_verse, imp_conf, imp_correct, imp_top3 = None, 0, False, []
        
        results_comparison.append({
            'file': file,
            'actual': actual_verse,
            'original_pred': orig_verse,
            'original_conf': orig_conf,
            'original_correct': orig_correct,
            'improved_pred': imp_verse,
            'improved_conf': imp_conf,
            'improved_correct': imp_correct,
            'improved_top3': imp_top3
        })
    
    # Calculate statistics
    orig_accuracy = sum([r['original_correct'] for r in results_comparison]) / len(results_comparison)
    imp_accuracy = sum([r['improved_correct'] for r in results_comparison]) / len(results_comparison)
    
    orig_avg_conf = np.mean([r['original_conf'] for r in results_comparison if r['original_conf'] > 0])
    imp_avg_conf = np.mean([r['improved_conf'] for r in results_comparison if r['improved_conf'] > 0])
    
    print(f"\n📊 HASIL PERBANDINGAN:")
    print(f"{'Model':<15} {'Accuracy':<12} {'Avg Confidence':<15}")
    print("-" * 42)
    print(f"{'Original':<15} {orig_accuracy:.2%}      {orig_avg_conf:.3f}")
    print(f"{'Improved':<15} {imp_accuracy:.2%}      {imp_avg_conf:.3f}")
    
    improvement = imp_accuracy - orig_accuracy
    print(f"\n🎯 Improvement: {improvement:.2%}")
    
    # Show detailed results
    print(f"\n📋 Detailed Results:")
    for result in results_comparison:
        actual_name = get_verse_name(result['actual'])
        orig_status = "✅" if result['original_correct'] else "❌"
        imp_status = "✅" if result['improved_correct'] else "❌"
        
        print(f"{result['file']}: {actual_name}")
        print(f"  Original: {orig_status} {get_verse_name(result['original_pred']) if result['original_pred'] is not None else 'Failed'} ({result['original_conf']:.2f})")
        print(f"  Improved: {imp_status} {get_verse_name(result['improved_pred']) if result['improved_pred'] is not None else 'Failed'} ({result['improved_conf']:.2f})")
        
        if result['improved_top3']:
            top3_str = ", ".join([f"{get_verse_name(v)}({p:.2f})" for v, p in result['improved_top3']])
            print(f"  Top-3: {top3_str}")
        print()
    
    return results_comparison

# Test improved model jika sudah di-train
print("🧪 TESTING IMPROVED MODEL")
print("=" * 40)

# Coba load improved model
improved_model_loaded, improved_encoder_loaded, improved_metadata = load_improved_model()

if improved_model_loaded is not None:
    print(f"✅ Improved model berhasil di-load!")
    print(f"📊 Model info:")
    print(f"   - Accuracy: {improved_metadata['final_metrics']['test_accuracy']:.4f}")
    print(f"   - F1 Score: {improved_metadata['final_metrics']['test_f1']:.4f}")
    print(f"   - Top-3 Accuracy: {improved_metadata['final_metrics']['top3_accuracy']:.4f}")
    
    # Test dengan file test.mp3 jika ada
    test_file_path = "test.mp3"
    if os.path.exists(test_file_path):
        print(f"\n🎵 Testing dengan file: {test_file_path}")
        improved_result = predict_with_improved_model(improved_model_loaded, improved_encoder_loaded, test_file_path)
        
        if improved_result:
            print(f"📝 Prediksi: {get_verse_name(improved_result['verse_number'])}")
            print(f"📊 Confidence: {improved_result['confidence']:.3f}")
            print(f"🥇 Top-3 predictions:")
            for i, (verse, prob) in enumerate(improved_result['top3_predictions'], 1):
                print(f"   {i}. {get_verse_name(verse)}: {prob:.3f}")
    
    # Perbandingan dengan model original jika tersedia
    if 'saved_model' in locals() and 'saved_label_encoder' in locals():
        print(f"\n🔄 Comparing with original model...")
        comparison = compare_models_performance(
            saved_model, improved_model_loaded,
            saved_label_encoder, improved_encoder_loaded,
            r"d:\new_project\quran_detect\sample_1"
        )
else:
    print("❌ Improved model belum di-train. Jalankan cell training terlebih dahulu!")